# Sentiment Analysis


1.   NLTK- Sentiment Intensity Analysis
2.   EmoLex



[Reference page for sentiment analysis](https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4)







## First, I need toimport the narratives.csv.

In [1]:
# !pip install swifter

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import swifter
import gc
import nltk 
import sklearn 
import collections
import sys
import itertools
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
narratives = pd.read_csv("/content/drive/MyDrive/QMSS Practicum 2021/narratives_token.csv")

In [5]:
narratives= narratives[['Date_received', 'Product', 'Sub_product', 'Issue',
       'Sub-issue', 'Consumer_complaint_narrative', 'Company_public_response',
       'Company', 'State', 'Tags', 'Consumer_consent_provided?',
       'Date_sent_to_company', 'Company_response_to_consumer',
       'Timely_response?', 'Consumer_disputed?',
       'Consumer_complaint_narrative_token',
       'Consumer_complaint_narrative_length']]

In [6]:
narratives.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld system inc tri collect debt mine ow...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN,"['transworld', 'system', 'inc', 'tri', 'collec...",9
1,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,past week receiv excess amount telephon call c...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN,"['past', 'week', 'receiv', 'excess', 'amount',...",29
2,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer commit sever feder violat privat law a...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN,"['pioneer', 'commit', 'sever', 'feder', 'viola...",72
3,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,employe work region accept keep call repeat ca...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN,"['employe', 'work', 'region', 'accept', 'keep'...",24
4,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previous request experian send copi verifi pro...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN,"['previous', 'request', 'experian', 'send', 'c...",74


## Choose CITIBANK, N.A. and for its first 200 complaints on Credit Card

Now, I will choose a company, which has enough complaints to analyze, since there is always an time-out warning, I will only focus on one single company : CITIBANK, N.A. and for its first 200 complaints



```
I will take CITI BANK as the target company to analyze
```



In [7]:
narratives['Company'].value_counts()

EQUIFAX, INC.                             77216
Experian Information Solutions Inc.       73507
TRANSUNION INTERMEDIATE HOLDINGS, INC.    71201
CITIBANK, N.A.                            21019
BANK OF AMERICA, NATIONAL ASSOCIATION     19912
                                          ...  
COMPASS MORTGAGE, INC.                        1
DARCARS AUTOMOTIVE  (Mariam Inc.)             1
Student Aid Experts                           1
Keys Auto Finance Inc                         1
Interest Smart Home Loans                     1
Name: Company, Length: 4806, dtype: int64

In [8]:
# set the company name to be CITIANK

CITIBANK_ = narratives[narratives['Company'].isin(['CITIBANK, N.A.'])==True]
CITIBANK_ = CITIBANK_[['Product', 'Consumer_complaint_narrative', 
                                 'Consumer_complaint_narrative_token', 'Consumer_complaint_narrative_length','Company_response_to_consumer']]
CITIBANK_.head()

,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer
102,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief
136,Checking or savings account,continu cfpb case particip second quater promo...,"['continu', 'cfpb', 'case', 'particip', 'secon...",87,Closed with monetary relief
150,Checking or savings account,small busi check save corpor account citibank ...,"['small', 'busi', 'check', 'save', 'corpor', '...",66,Closed with explanation
243,"Credit reporting, credit repair services, or o...",first credit card sear maci best buy suddenli ...,"['first', 'credit', 'card', 'sear', 'maci', 'b...",59,Closed with explanation
291,"Credit reporting, credit repair services, or o...",cbna inquiri date cbna inquiri date cbna inqui...,"['cbna', 'inquiri', 'date', 'cbna', 'inquiri',...",29,Closed with non-monetary relief


In [9]:
# select product for credit card

CITIBANK_credit_card= CITIBANK_.loc[CITIBANK_['Product'] == 'Credit card']
CITIBANK_credit_card.head()


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer
102,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief
89013,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation
162316,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation
193168,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief
243068,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation


In [10]:
# only take the first 200 rows for a smooth run
CITIBANK_credit_card_subset=CITIBANK_credit_card[0:200]
CITIBANK_credit_card_subset.reset_index(drop=True, inplace=True)
CITIBANK_credit_card_subset.head()

,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation


## NLTK.Sentiment Intensity Analyzer

In [11]:
  >>> import nltk
  >>> nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [12]:
# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
CITIBANK_credit_card_subset['NLTK_sentiments'] = CITIBANK_credit_card_subset['Consumer_complaint_narrative'].apply(lambda x: sid.polarity_scores(x))

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
CITIBANK_credit_card_subset.head()

,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co..."
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co..."
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co..."
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com..."
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co..."


### From the sentiment score above, we can see that the nltk.sentiment_intensity_analyzer does not do well on giving a sentiment score. 

## Sentiment Analysis w/ EmoLex

[The reference page for EmoLex](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm)

In [14]:
CITIBANK_narr_credit_card_subset_token = CITIBANK_credit_card_subset
CITIBANK_narr_credit_card_subset_token.head()

,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co..."
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co..."
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co..."
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com..."
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co..."


In [15]:
# Make data directory if it doesn't exist
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/upshot-trump-emolex/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt -P data

--2021-02-24 22:17:46--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/upshot-trump-emolex/data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2581050 (2.5M) [text/plain]
Saving to: ‘data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt’

NRC-emotion-lexicon 100%[===================>]   2.46M  2.12MB/s    in 1.2s    

2021-02-24 22:17:48 (2.12 MB/s) - ‘data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt’ saved [2581050/2581050]



In [16]:
filepath = "data/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t', keep_default_na=False)
emolex_df = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_df.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,aback,0,0,0,0,0,0,0,0,0,0
1,abacus,0,0,0,0,0,0,0,0,0,1
2,abandon,0,0,0,1,0,1,0,1,0,0
3,abandoned,1,0,0,1,0,1,0,1,0,0
4,abandonment,1,0,0,1,0,1,0,1,1,0


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(vocabulary=emolex_df.word,
                      use_idf=False, 
                      norm='l1')

In [18]:
matrix = vec.fit_transform(CITIBANK_narr_credit_card_subset_token.Consumer_complaint_narrative_token)
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

,aback,abacus,abandon,abandoned,abandonment,abate,abatement,abba,abbot,abbreviate,abbreviation,abdomen,abdominal,abduction,aberrant,aberration,abeyance,abhor,abhorrent,abide,ability,abject,ablation,ablaze,abnormal,aboard,abode,abolish,abolition,abominable,abomination,aboriginal,abort,abortion,abortive,abound,abovementioned,abrasion,abroad,abrogate,...,yearly,yearn,yearning,years,yeast,yell,yellow,yellows,yelp,yeoman,yesterday,yesteryear,yew,yield,yielding,yogi,yoke,yolk,yon,yonder,young,younger,youth,zany,zap,zeal,zealot,zealous,zebra,zenith,zephyr,zeppelin,zest,zip,zodiac,zone,zoo,zoological,zoology,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
negative_words = emolex_df[emolex_df.negative == 1].word

CITIBANK_narr_credit_card_subset_token['Emolex_negative'] = wordcount_df[negative_words].sum(axis=1)
CITIBANK_narr_credit_card_subset_token.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391


In [20]:
positive_words = emolex_df[emolex_df.positive == 1].word

CITIBANK_narr_credit_card_subset_token['Emolex_positive'] = wordcount_df[positive_words].sum(axis=1)
CITIBANK_narr_credit_card_subset_token.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913


In [21]:
angry_words = emolex_df[emolex_df.anger == 1]['word']
CITIBANK_narr_credit_card_subset_token['Emolex_anger'] = wordcount_df[angry_words].sum(axis=1)
CITIBANK_narr_credit_card_subset_token.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive,Emolex_anger
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725,0.021978
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000,0.000000
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143,0.000000
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000,0.080000
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913,0.043478


In [22]:
sadness_words = emolex_df[emolex_df.sadness == 1]['word']
CITIBANK_narr_credit_card_subset_token['Emolex_sadness'] = wordcount_df[sadness_words].sum(axis=1)
CITIBANK_narr_credit_card_subset_token.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive,Emolex_anger,Emolex_sadness
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725,0.021978,0.054945
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000,0.000000,0.250000
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143,0.000000,0.035714
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000,0.080000,0.000000
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913,0.043478,0.217391
5,Credit card,charg total sinc bank citibank never agre beli...,"['charg', 'total', 'sinc', 'bank', 'citibank',...",70,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.849, 'pos': 0.102, 'co...",0.030303,0.181818,0.000000,0.090909
6,Credit card,maci credit card want place order never past i...,"['maci', 'credit', 'card', 'want', 'place', 'o...",56,Closed with explanation,"{'neg': 0.044, 'neu': 0.778, 'pos': 0.178, 'co...",0.074074,0.185185,0.000000,0.000000
7,Credit card,took manag credit card institut auto enrol cre...,"['took', 'manag', 'credit', 'card', 'institut'...",48,Closed with monetary relief,"{'neg': 0.1, 'neu': 0.6, 'pos': 0.3, 'compound...",0.222222,0.296296,0.074074,0.074074
8,Credit card,fraudul charg account declar fraud card close ...,"['fraudul', 'charg', 'account', 'declar', 'fra...",52,Closed with monetary relief,"{'neg': 0.129, 'neu': 0.833, 'pos': 0.038, 'co...",0.115385,0.038462,0.115385,0.038462
9,Credit card,citicard list credit account credit report acc...,"['citicard', 'list', 'credit', 'account', 'cre...",9,Closed with explanation,"{'neg': 0.253, 'neu': 0.4, 'pos': 0.347, 'comp...",0.125000,0.250000,0.125000,0.000000


## TextBlob: Simple rule-based API for sentiment analysis

In [23]:
import re
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [24]:
CITIBANK_narr_credit_card_subset_token['TextBlob_sentiment'] = CITIBANK_narr_credit_card_subset_token['Consumer_complaint_narrative'].apply(lambda narrative: TextBlob(narrative).sentiment)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
CITIBANK_narr_credit_card_subset_token['TextBlob_polarity'] = CITIBANK_narr_credit_card_subset_token['TextBlob_sentiment'].apply(lambda sentiment: sentiment[0])
CITIBANK_narr_credit_card_subset_token['TextBlob_subjectivity'] = CITIBANK_narr_credit_card_subset_token['TextBlob_sentiment'].apply(lambda sentiment: sentiment[1])

CITIBANK_narr_credit_card_subset_token.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive,Emolex_anger,Emolex_sadness,TextBlob_sentiment,TextBlob_polarity,TextBlob_subjectivity
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725,0.021978,0.054945,"(0.08218763994626063, 0.543951833607006)",0.082188,0.543952
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000,0.000000,0.250000,"(0.0, 0.0)",0.000000,0.000000
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143,0.000000,0.035714,"(0.175, 0.30833333333333335)",0.175000,0.308333
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000,0.080000,0.000000,"(-0.275, 0.6666666666666666)",-0.275000,0.666667
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913,0.043478,0.217391,"(0.2857142857142857, 0.5357142857142857)",0.285714,0.535714


In [26]:
# CITIBANK_narr_credit_card_subset_token.to_csv('/content/drive/MyDrive/QMSS Practicum 2021/CITIBANK_narr_credit_card_subset_token.csv')

## Summary of the Sentiment Analysis result

#### I am trying best to take out all the neg_score

In [ ]:
CITIBANK_narr_credit_card_subset_token.head(10)

,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive,Emolex_anger,Emolex_sadness,TextBlob_sentiment,TextBlob_polarity,TextBlob_subjectivity,NLTK_neg_score
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725,0.021978,0.054945,"(0.08218763994626063, 0.543951833607006)",0.082188,0.543952,None
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000,0.000000,0.250000,"(0.0, 0.0)",0.000000,0.000000,None
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143,0.000000,0.035714,"(0.175, 0.30833333333333335)",0.175000,0.308333,None
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000,0.080000,0.000000,"(-0.275, 0.6666666666666666)",-0.275000,0.666667,None
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913,0.043478,0.217391,"(0.2857142857142857, 0.5357142857142857)",0.285714,0.535714,None
5,Credit card,charg total sinc bank citibank never agre beli...,"['charg', 'total', 'sinc', 'bank', 'citibank',...",70,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.849, 'pos': 0.102, 'co...",0.030303,0.181818,0.000000,0.090909,"(0.1425, 0.3633333333333334)",0.142500,0.363333,None
6,Credit card,maci credit card want place order never past i...,"['maci', 'credit', 'card', 'want', 'place', 'o...",56,Closed with explanation,"{'neg': 0.044, 'neu': 0.778, 'pos': 0.178, 'co...",0.074074,0.185185,0.000000,0.000000,"(-0.06875, 0.259375)",-0.068750,0.259375,None
7,Credit card,took manag credit card institut auto enrol cre...,"['took', 'manag', 'credit', 'card', 'institut'...",48,Closed with monetary relief,"{'neg': 0.1, 'neu': 0.6, 'pos': 0.3, 'compound...",0.222222,0.296296,0.074074,0.074074,"(-0.2752840909090909, 0.640625)",-0.275284,0.640625,None
8,Credit card,fraudul charg account declar fraud card close ...,"['fraudul', 'charg', 'account', 'declar', 'fra...",52,Closed with monetary relief,"{'neg': 0.129, 'neu': 0.833, 'pos': 0.038, 'co...",0.115385,0.038462,0.115385,0.038462,"(0.10227272727272727, 0.46590909090909094)",0.102273,0.465909,None
9,Credit card,citicard list credit account credit report acc...,"['citicard', 'list', 'credit', 'account', 'cre...",9,Closed with explanation,"{'neg': 0.253, 'neu': 0.4, 'pos': 0.347, 'comp...",0.125000,0.250000,0.125000,0.000000,"(0.0, 0.0)",0.000000,0.000000,None


In [ ]:
# CITIBANK_narr_credit_card_subset_token['NLTK_sentiments'].get('neg')

In [ ]:
CITIBANK_narr_credit_card_subset_token['NLTK_neg_score'] = CITIBANK_narr_credit_card_subset_token['NLTK_sentiments'].get("neg")
CITIBANK_narr_credit_card_subset_token.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Product,Consumer_complaint_narrative,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length,Company_response_to_consumer,NLTK_sentiments,Emolex_negative,Emolex_positive,Emolex_anger,Emolex_sadness,TextBlob_sentiment,TextBlob_polarity,TextBlob_subjectivity,NLTK_neg_score
0,Credit card,stupid enough charg item maci maci credit card...,"['stupid', 'enough', 'charg', 'item', 'maci', ...",226,Closed with monetary relief,"{'neg': 0.049, 'neu': 0.762, 'pos': 0.188, 'co...",0.109890,0.274725,0.021978,0.054945,"(0.08218763994626063, 0.543951833607006)",0.082188,0.543952,None
1,Credit card,tri resolv problem citibank andtheir credit ca...,"['tri', 'resolv', 'problem', 'citibank', 'andt...",21,Closed with explanation,"{'neg': 0.182, 'neu': 0.505, 'pos': 0.313, 'co...",0.250000,0.250000,0.000000,0.250000,"(0.0, 0.0)",0.000000,0.000000,None
2,Credit card,full time employ per annum credit score father...,"['full', 'time', 'employ', 'per', 'annum', 'cr...",41,Closed with explanation,"{'neg': 0.075, 'neu': 0.653, 'pos': 0.273, 'co...",0.035714,0.357143,0.000000,0.035714,"(0.175, 0.30833333333333335)",0.175000,0.308333,None
3,Credit card,sign maci credit card time maci enrol credit p...,"['sign', 'maci', 'credit', 'card', 'time', 'ma...",51,Closed with monetary relief,"{'neg': 0.028, 'neu': 0.751, 'pos': 0.22, 'com...",0.080000,0.240000,0.080000,0.000000,"(-0.275, 0.6666666666666666)",-0.275000,0.666667,None
4,Credit card,citi thank prefer credit card year somehow dec...,"['citi', 'thank', 'prefer', 'credit', 'card', ...",39,Closed with explanation,"{'neg': 0.191, 'neu': 0.652, 'pos': 0.157, 'co...",0.217391,0.173913,0.043478,0.217391,"(0.2857142857142857, 0.5357142857142857)",0.285714,0.535714,None
